In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Find the dataset with model @ - https://www.kaggle.com/ash1706/ner-model

This notebook trains a NER BERT BASE model 
Credits to https://www.kaggle.com/tungmphung/pytorch-bert-for-named-entity-recognition fro such a amazing notebook
and all the owners of the dataset i am using
also a great NER data processing notebook - https://www.kaggle.com/devashishprasad/bio-labeled-dataset

In [ ]:
!pip install datasets --no-index --find-links=file:///kaggle/input/coleridge-packages/packages/datasets
!pip install ../input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
!pip install ../input/coleridge-packages/tokenizers-0.10.1-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/coleridge-packages/transformers-4.5.0.dev0-py3-none-any.whl

Lets load the NER labeled data set

In [ ]:
import pickle
import pandas as pd
import numpy as np

X = []
y = []

with open('../input/test-labels-ner/sentences.pkl', 'rb') as f:
    X = pickle.load(f)
    
with open('../input/test-labels-ner/bio_labels.pkl', 'rb') as f:
    y = pickle.load(f)


In [ ]:
import os
import re
import json
import time
import datetime
import random
import glob
import importlib

import numpy as np
import pandas as pd

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

random.seed(123)
np.random.seed(456)

In [ ]:
!cp /kaggle/input/coleridge-packages/my_seqeval.py ./

In [ ]:
MAX_LENGTH = 64 # max no. words for each sentence.
OVERLAP = 20 # if a sentence exceeds MAX_LENGTH, we split it to multiple sentences with overlapping

MAX_SAMPLE = None # set a small number for experimentation, set None for production.

In [ ]:
train_path = '../input/coleridgeinitiative-show-us-the-data/train.csv'
paper_train_folder = '../input/coleridgeinitiative-show-us-the-data/train'

train = pd.read_csv(train_path)
train = train[:MAX_SAMPLE]
print(f'No. raw training rows: {len(train)}')

In [ ]:
train = train.groupby('Id').agg({
    'pub_title': 'first',
    'dataset_title': '|'.join,
    'dataset_label': '|'.join,
    'cleaned_label': '|'.join
}).reset_index()

print(f'No. grouped training rows: {len(train)}')

In [ ]:
# papers = {}
# for paper_id in train['Id'].unique():
#     with open(f'{paper_train_folder}/{paper_id}.json', 'r') as f:
#         paper = json.load(f)
#         papers[paper_id] = paper

In [ ]:
c=0
co = 0

you can play around with how much data you want to use i would suggest to use all of it but then again i was testing this approach so using all of it didnt make sense as it would require a lot of time and resources

# use version 8 for a bigger sample

In [ ]:
with open('train_ner.json', 'w') as f:
    for i in range(len(X)):
        words = X[i]
        nes = y[i]
#         row_json = {'tokens' : words, 'tags' : nes}
#         co = co+1
#         json.dump(row_json, f)
#         f.write('\n')
        if('B' in nes):
            row_json = {'tokens' : words, 'tags' : nes}
            co = co+1
            json.dump(row_json, f)
            f.write('\n')
#         else:
#             c = c+1
#             if(c>100):
#                 c=0
#                 co=co+1
#                 row_json = {'tokens' : words, 'tags' : nes}
#                 json.dump(row_json, f)
#                 f.write('\n')
                
                

In [ ]:
del X
del y
# will run into memory issues otherwise

In [ ]:
print(co)

In [ ]:
# len(X)

In [ ]:
!pip install --upgrade git+https://github.com/intake/filesystem_spec
# comment out if you want to run without internet

Training begins

In [ ]:
!python ../input/kaggle-ner-utils/kaggle_run_ner.py \
--model_name_or_path 'bert-base-cased' \
--train_file './train_ner.json' \
--validation_file './train_ner.json' \
--num_train_epochs 1  \
--per_device_train_batch_size 8 \
--per_device_eval_batch_size 8 \
--save_steps 15000 \
--output_dir './output' \
--report_to 'none' \
--seed 123 \
--do_train